# 訓練圖片轉換

#將訓練圖片合併、調整大小成1024*512保存後，分別移動到river資料夾與road資料夾

In [2]:
import os
from PIL import Image
import shutil

# 設定原始圖片和標籤圖片的路徑
img_folder_path = r"D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\35_Competition 2_Training dataset_V3\Training_dataset\img"
label_img_folder_path = r"D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\35_Competition 2_Training dataset_V3\Training_dataset\label_img"

# 分類文件夾路徑設置
road_folder = r'D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\train_road\train'
river_folder = r'D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\train_river\train'

# 順便複製 train_river\train 的所有圖片到 Real-ESRGAN-master\inputs 中，為後續的圖片處理做準備
esrgan_inputs_folder = r'D:\AICUP\Real-ESRGAN-master\inputs'


# 確保分類文件夾存在
os.makedirs(road_folder, exist_ok=True)
os.makedirs(river_folder, exist_ok=True)

# 獲取圖片名稱列表
img_files = os.listdir(img_folder_path)
label_img_files = os.listdir(label_img_folder_path)

# 確認兩個目錄下的圖片名稱一一對應
assert set(i.split('.')[0] for i in img_files) == set(i.split('.')[0] for i in label_img_files)

# 遍歷所有圖片
for img_file in img_files:
    # 找到對應的 label_img
    label_img_file = img_file.split('.')[0] + '.png'
    
    # 打開這兩張圖片
    img_path = os.path.join(img_folder_path, img_file)
    img = Image.open(img_path)
    label_img_path = os.path.join(label_img_folder_path, label_img_file)
    label_img = Image.open(label_img_path)
    
    # 確認圖片大小，並創建新的合併圖片
    width, height = max(img.size[0], label_img.size[0]), max(img.size[1], label_img.size[1])
    new_img = Image.new('RGB', (2 * width, height))
    
    # 將這兩張圖片左右拼接
    new_img.paste(img, (0, 0))
    new_img.paste(label_img, (width, 0))
    
    # 調整圖片尺寸
    resized_image = new_img.resize((1024, 512), Image.LANCZOS)
    
    # 根據文件名決定保存路徑並保存圖片
    if img_file.startswith('TRA_RO'):
        save_path = os.path.join(road_folder, img_file)
    elif img_file.startswith('TRA_RI'):
        save_path = os.path.join(river_folder, img_file)
    else:
        continue  # 如果文件名不符合規則，則跳過
    
    # 保存調整後的圖片
    resized_image.save(save_path)



# 將 river_folder 中的所有圖片複製到 esrgan_inputs_folder 中
for river_img in os.listdir(river_folder):
    river_img_path = os.path.join(river_folder, river_img)
    shutil.copy(river_img_path, esrgan_inputs_folder)

print("圖片合併和分類完成，並已將 river 圖片複製到 Real-ESRGAN 的 inputs 文件夾中")

圖片合併和分類完成，並已將 river 圖片複製到 Real-ESRGAN 的 inputs 文件夾中


# 測試圖片轉換

In [1]:
import os
from PIL import Image

# 資料夾路徑設定
public_img_folder = r'D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\35_Competition 2_public testing dataset\img'
private_img_folder = r'D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\35_Competition 2_Private Test Dataset\img'

# 分類文件夾路徑設置
river_folder = r'D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\pub_and_pri_test_river\test'
road_folder = r'D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\pub_and_pri_test_road\test'

# 確保目標文件夾存在
os.makedirs(river_folder, exist_ok=True)
os.makedirs(road_folder, exist_ok=True)

# 處理函數
def process_images(src_folder, dst_folder_road, dst_folder_river):
    for filename in os.listdir(src_folder):
        base, ext = os.path.splitext(filename)
        if ext.lower() not in ['.jpg', '.png', '.jpeg']:
            continue  # Skip non-image files
        
        # 讀取原始圖片
        img_path = os.path.join(src_folder, filename)
        img = Image.open(img_path)
        
        # 創建新的圖片，尺寸為原圖兩倍寬度
        new_img = Image.new('RGB', (img.width * 2, img.height))
        new_img.paste(img, (0, 0))  # 左側貼上原圖
        new_img.paste(img, (img.width, 0))  # 右側貼上原圖
        
        # 調整圖片大小
        new_img = new_img.resize((1024, 512), Image.LANCZOS)
        
        # 根據圖片開頭決定保存路徑
        if filename.startswith(('PRI_RO', 'PUB_RO')):
            new_img.save(os.path.join(dst_folder_road, filename))
        elif filename.startswith(('PRI_RI', 'PUB_RI')):
            new_img.save(os.path.join(dst_folder_river, filename))

# 處理公開測試集
process_images(public_img_folder, road_folder, river_folder)

# 處理不公開測試集
process_images(private_img_folder, road_folder, river_folder)

print("圖片處理完畢，複製自己一份並分類到指定資料夾中。")

圖片處理完畢，複製自己一份並分類到指定資料夾中。


# 將Real-ESRGAN的outputs中的圖片後綴移除並移到pix2pix的datasets\train_river_Real_ESRGAN\train資料夾中

In [5]:
import os

# 設定原始資料夾路徑
folder_path = r"D:\AICUP\Real-ESRGAN-master\outputs"
# 設定目標資料夾路徑
target_folder_path = r"D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\train_river_Real_ESRGAN\train"

# 確保目標文件夾存在
os.makedirs(target_folder_path, exist_ok=True)

# 取得資料夾中的所有檔案名稱
files = os.listdir(folder_path)

# 遍歷所有檔案
for filename in files:
    # 如果檔案名稱包含 `__`，則進行替換
    if '__' in filename:
        new_filename = filename.replace('__', '')
    else:
        new_filename = filename
    # 取得完整的原檔案路徑
    old_file = os.path.join(folder_path, filename)
    # 指定新檔案路徑到目標資料夾
    new_file = os.path.join(target_folder_path, new_filename)
    # 移動檔案到新位置
    os.rename(old_file, new_file)

print("所有圖片名稱已成功更新並移動到指定資料夾。")

所有圖片名稱已成功更新並移動到指定資料夾。


# 將Real-ESRGAN的outputs中的圖片後綴移除並移到pix2pix的datasets\pub_and_pri_test_river_Real_ESRGAN\test資料夾中

In [6]:
import os

# 設定原始資料夾路徑
folder_path = r"D:\AICUP\Real-ESRGAN-master\outputs"
# 設定目標資料夾路徑
target_folder_path = r"D:\AICUP\AI_cup_demo_code\pytorch-CycleGAN-and-pix2pix-master\datasets\pub_and_pri_test_river_Real_ESRGAN\test"

# 確保目標文件夾存在
os.makedirs(target_folder_path, exist_ok=True)

# 取得資料夾中的所有檔案名稱
files = os.listdir(folder_path)

# 遍歷所有檔案
for filename in files:
    # 如果檔案名稱包含 `__`，則進行替換
    if '__' in filename:
        new_filename = filename.replace('__', '')
    else:
        new_filename = filename
    # 取得完整的原檔案路徑
    old_file = os.path.join(folder_path, filename)
    # 指定新檔案路徑到目標資料夾
    new_file = os.path.join(target_folder_path, new_filename)
    # 移動檔案到新位置
    os.rename(old_file, new_file)

print("所有圖片名稱已成功更新並移動到指定資料夾。")

所有圖片名稱已成功更新並移動到指定資料夾。


# 提取出生圖片並改名且resize放到upload資料夾中

In [7]:
from PIL import Image
import os

# 開始的文件夾路徑列表
start_dirs = [
    r'D:\AICUP\AI_cup_demo_code\results\ROAD_pix2pix_river18\ROAD_pix2pix_river18\test_latest\images',
    r'D:\AICUP\AI_cup_demo_code\results\ROAD_pix2pix_road17\ROAD_pix2pix_road17\test_latest\images'
]

# 目標文件夾路徑
target_dir = r'D:\AICUP\AI_cup_demo_code\results\upload'

# 目標圖像尺寸
target_width = 428
target_height = 240

# 如果目標文件夾不存在，則創建它
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# 遍歷每個開始的文件夾
for start_dir in start_dirs:
    for filename in os.listdir(start_dir):
        # 檢查文件名是否含 '_fake_B' 並且是 PNG 格式
        if '_fake_B' in filename and filename.endswith('.png'):
            new_name = filename.replace('_fake_B', '')
            start_file_path = os.path.join(start_dir, filename)
            target_file_path = os.path.join(target_dir, new_name)
            
            # 讀取圖像
            with Image.open(start_file_path) as img:
                # 調整圖像大小
                resized_img = img.resize((target_width, target_height))
                
                # 保存圖像為 PNG 格式
                resized_img.save(target_file_path, 'PNG')
            
            print(f"Processed: {filename} -> {new_name}")

print("所有圖片處理完畢並保存至指定目錄。")

Processed: PRI_RI_2000000_fake_B.png -> PRI_RI_2000000.png
Processed: PRI_RI_2000001_fake_B.png -> PRI_RI_2000001.png
Processed: PRI_RI_2000002_fake_B.png -> PRI_RI_2000002.png
Processed: PRI_RI_2000003_fake_B.png -> PRI_RI_2000003.png
Processed: PRI_RI_2000004_fake_B.png -> PRI_RI_2000004.png
Processed: PRI_RI_2000005_fake_B.png -> PRI_RI_2000005.png
Processed: PRI_RI_2000006_fake_B.png -> PRI_RI_2000006.png
Processed: PRI_RO_2000360_fake_B.png -> PRI_RO_2000360.png
Processed: PRI_RO_2000361_fake_B.png -> PRI_RO_2000361.png
Processed: PRI_RO_2000362_fake_B.png -> PRI_RO_2000362.png
Processed: PRI_RO_2000363_fake_B.png -> PRI_RO_2000363.png
Processed: PRI_RO_2000364_fake_B.png -> PRI_RO_2000364.png
Processed: PRI_RO_2000365_fake_B.png -> PRI_RO_2000365.png
Processed: PRI_RO_2000366_fake_B.png -> PRI_RO_2000366.png
Processed: PRI_RO_2000367_fake_B.png -> PRI_RO_2000367.png
Processed: PRI_RO_2000368_fake_B.png -> PRI_RO_2000368.png
Processed: PRI_RO_2000369_fake_B.png -> PRI_RO_2000369.p